In [9]:
import json
import pandas as pd
from bokeh.io import show, export_png, save, output_file
from bokeh.models import (CDSView, ColorBar, ColumnDataSource,
                          CustomJS, CustomJSFilter, 
                          GeoJSONDataSource, HoverTool,
                          LinearColorMapper, Slider)
from bokeh.layouts import column, row, widgetbox
from bokeh.palettes import brewer
from bokeh.plotting import figure
#load data frame made with add_data notebook
#plot_data=pd.read_csv('datatovisualize.csv')
plot_data=pd.read_csv('coviddata.csv')
# Input GeoJSON source that contains features for plotting
#will be hard to do multiple cuz this source needs a new name each time
source1 = GeoJSONDataSource(geojson = plot_data.to_json())

In [13]:
# Create figure object.
p1 = figure(title = 'Total COVID Cases', 
           plot_height = 600 ,
           plot_width = 950, 
           toolbar_location = 'below',
           tools = 'pan, wheel_zoom, box_zoom, reset')
p1.xgrid.grid_line_color = None
p1.ygrid.grid_line_color = None
# Add patch renderer to figure.
states = p1.patches('xs','ys', source = source1,
                   fill_color = None,
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)
# Create hover tool, this would be adding interaction
p1.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@NAME'),
                                ('Total Covid Cases','@{Total Cases}'),
                                 ('Total Covid Deaths', "@{Total Deaths}")]))
#I'm also having trouble visualizing and exporting the figures
output_file("figure.html", title='Bokeh Plot', mode=None, root_dir=None)
save(p1)

RuntimeError: Models must be owned by only a single document, Selection(id='1508', ...) is already in a doc

In [6]:
source2 = GeoJSONDataSource(geojson = plot_data.to_json())
# Define color palettes
palette = brewer['BuGn'][8]
palette = palette[::-1] # reverse order of colors so higher values have darker colors
# Instantiate LinearColorMapper that linearly maps numbers in a range, into a sequence of colors.
color_mapper = LinearColorMapper(palette = palette, low = 0, high = 5000)
# Define custom tick labels for color bar.
tick_labels = {'0': '0', '0': '10000','10,000'
 '20000':'20,000', '30000':'30,000',
 '40000':'40,000', '50000':'50,000',
 '60000':'60,000', '70000':'70,000',
 '90000':'90,000+'}
# Create color bar.
color_bar = ColorBar(color_mapper = color_mapper, 
                     label_standoff = 8,
                     width = 500, height = 20,
                     border_line_color = None,
                     location = (0,0), 
                     orientation = 'horizontal',
                     major_label_overrides = tick_labels)
# Create figure object.
p = figure(title = 'Covid Cases and Deaths', 
           plot_height = 600, plot_width = 950, 
           toolbar_location = 'below',
           tools = 'pan, wheel_zoom, box_zoom, reset')
p.xgrid.grid_line_color = None
p.ygrid.grid_line_color = None
# Add patch renderer to figure.
states = p.patches('xs','ys', source = source2,
                   fill_color = {'field' :'Case Rate per 100000',
                                 'transform' : color_mapper},
                   line_color = 'gray', 
                   line_width = 0.25, 
                   fill_alpha = 1)
# Create hover tool, this would be adding interaction
p.add_tools(HoverTool(renderers = [states],
                      tooltips = [('State','@NAME'),
                               ('Total Covid Cases','@{Total Cases}'),
                                 ('Total Covid Deaths', "@{Total Deaths}"),
                                 ('Case Rate per 100,000' ,"@{Case Rate per 100000}")]))
# Specify layout
p.add_layout(color_bar, 'below')
show(p)